In [2]:
!pip install tflite_runtime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import tflite_runtime.interpreter as tflite

print(tflite.Interpreter)

<class 'tflite_runtime.interpreter.Interpreter'>


In [4]:
from PIL import Image
import numpy as np
import time
import matplotlib.pyplot as plt

In [5]:
interpreter = tflite.Interpreter("/content/drive/MyDrive/Jugend Forscht/models/model.tflite")

In [6]:
interpreter.allocate_tensors()

In [7]:
from pprint import pprint
input_details = interpreter.get_input_details()
pprint(input_details)

[{'dtype': <class 'numpy.uint8'>,
  'index': 0,
  'name': 'input_1',
  'quantization': (0.003921568859368563, 0),
  'quantization_parameters': {'quantized_dimension': 0,
                              'scales': array([0.00392157], dtype=float32),
                              'zero_points': array([0], dtype=int32)},
  'shape': array([  1, 224, 224,   3], dtype=int32),
  'shape_signature': array([ -1, 224, 224,   3], dtype=int32),
  'sparsity_parameters': {}}]


In [13]:
def set_input_tensor(interpreter, image):
  input_tensor = interpreter.tensor(0)()[0]
  input_tensor[:, :] = image


In [11]:
def classify_image(interpreter, image):
  interpreter.invoke()
  output_details = interpreter.get_output_details()[0]
  scores = interpreter.get_tensor(output_details["index"])[0]
  print(np.max(np.unique(scores)))

  scale, zero_point = output_details["quantization"]
  scores_dequan = scale * (scores - zero_point)

  dequan_max_score = np.max(np.unique(scores_dequan))
  print(dequan_max_score)

  max_score_index = np.where(scores_dequan == np.max(np.unique(scores_dequan)))[0][0]

  print(max_score_index, dequan_max_score)



204
0.796875
0 0.796875


In [14]:
for c in ["b", "g", "r", "y"]:
  image = Image.open(f"/content/drive/MyDrive/Jugend Forscht/colors/{c}/0-5.jpg").resize((224, 224))
  set_input_tensor(interpreter, image)
  classify_image(interpreter, image)

204
0.796875
0 0.796875
227
0.88671875
1 0.88671875
224
0.875
3 0.875
199
0.77734375
4 0.77734375


In [16]:
!ls -lh "/content/drive/MyDrive/Jugend Forscht/models/model.tflite"

-rw------- 1 root root 3.9M Jan  3 19:28 '/content/drive/MyDrive/Jugend Forscht/models/model.tflite'
